In [ ]:
from ast import literal_eval
import pandas as pd
from tqdm.notebook import tqdm
import numpy as np
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns
class_ids = [6, 7, 8, 9, 10, 25, 41, 105, 110, 115, 148, 156, 222, 228, 235, 256, 
             280, 310, 387, 392, 394, 398, 401, 402, 430, 480, 485, 673]
name = ['大螟', '二化螟', '稻纵卷叶螟', '白背飞虱', '褐飞虱属', '地老虎', '蝼蛄', '粘虫', '草地螟', '甜菜夜蛾', '黄足猎蝽', '八点灰灯蛾', 
         '棉铃虫', '二点委夜蛾', '甘蓝夜蛾', '蟋蟀', '黄毒蛾', '稻螟蛉', '紫条尺蛾', '水螟蛾', '线委夜蛾', '甜菜白带野螟', '歧角螟', 
        '瓜绢野螟', '豆野螟', '石蛾', '大黑鳃金龟', '干纹冬夜蛾']

N_FOLDS = 6
seed = 2022
TRAIN_PATH = '../input/tddatafinal/marge23'

In [ ]:
df1 = pd.read_csv('../input/taidicup2022dataset/TaiDiCup2022DataSet/图片虫子位置详情表.csv',encoding='gbk', index_col=0, 
                 names=['file_path', 'class_id','Chinese_name','x','y','xmin','ymin','xmax','ymax'],header=0)
df = pd.read_csv('../input/td-csv/finalout.csv',encoding='gbk', index_col=0, 
                 names=['file_path', 'class_id','xmin','ymin','xmax','ymax','label'],header=0)
empty_path = []
for i in range(len(df1)):
    if df1.iloc[i,1]==0:
        empty_path.append(df1.iloc[i,0])
df = df[df['label']==1].reset_index(drop = True)
# df = df[df['class_id']!=0].reset_index(drop = True)
for i in range(len(class_ids)):
    df.class_id.replace(class_ids[i], i, inplace=True)
        
df['fold'] = -1
strat_kfold = StratifiedKFold(n_splits=N_FOLDS, random_state=seed, shuffle=True)

for i, (_, train_index) in enumerate(strat_kfold.split(df.index, df['class_id'])):
    df.iloc[train_index, -1] = i
df['fold'] = df['fold'].astype('int')
df = df.sort_values(by='file_path', ascending=True).reset_index(drop = True)

last_file = ''
for i in range(len(df)):
    if df.iloc[i,0]!=last_file:
        fold_t = df.iloc[i,-1]
        last_file = df.iloc[i,0]
    else:
        df.iloc[i,-1]=fold_t

plt.figure(figsize=(8,6),dpi=300)
sns.jointplot(x='fold', y='class_id', data=df,
              kind='kde', fill=True, thresh=0, size=8, cmap='BuPu')#Pastel1_r
df.head(5)



In [ ]:
def coco(df):
    images = []
    annotations = []
    categories = []
    exist_class = []
    annotion_id = 0
    last_file = ''
    idx = -1
    for i in range(27):
        categories.append({
        'id':i,
        'name':str(i)
    })
    for i, row in tqdm(df.iterrows(), total = len(df)):
        if row['file_path'] in empty_path:
            width  = 5472/(3648/1280)
            height = 1280
        else:
            width  = 5472
            height = 3648
            
        if row['file_path']==last_file:
            xmin,ymin,xmax,ymax = row['xmin'],row['ymin'],row['xmax'],row['ymax']
            x = int(xmin)
            y = int(ymin)
            w = int(xmax)-int(xmin)
            h = int(ymax)-int(ymin)
            annotations.append({
                "id": annotion_id,
                "image_id": idx,
                "category_id": row['class_id'],
                "bbox": [x,y,w,h],
                "area": w*h,
                "segmentation": [],
                "iscrowd": 0
            })
            annotion_id += 1
        else:
            idx += 1
            last_file = row['file_path']
            images.append({
                "id": idx,
                "file_name": f"{row['file_path']}",
                "height": width,
                "width": height,
            })
            xmin,ymin,xmax,ymax = row['xmin'],row['ymin'],row['xmax'],row['ymax']
            x = int(xmin)
            y = int(ymin)
            w = int(xmax)-int(xmin)
            h = int(ymax)-int(ymin)
            annotations.append({
                "id": annotion_id,
                "image_id": idx,
                "category_id": row['class_id'],
                "bbox": [x,y,w,h],
                "area": w*h,
                "segmentation": [],
                "iscrowd": 0
            })
            annotion_id += 1
            

    json_file = {'categories':categories, 'images':images, 'annotations':annotations}
    return json_file

In [ ]:
json_train = coco(df[df['fold']!=0])
json_valid = coco(df[df['fold']==0])
# json_df = coco(df)

In [ ]:
!mkdir /kaggle/working/data

In [ ]:
import json

with open('/kaggle/working/data/train.json', 'w', encoding='utf-8') as f:
    json.dump(json_train, f, ensure_ascii=True, indent=4)
    
with open('/kaggle/working/data/valid.json', 'w', encoding='utf-8') as f:
    json.dump(json_valid, f, ensure_ascii=True, indent=4)

In [ ]:
import os
os.makedirs('/kaggle/working/data/train2017', exist_ok=True)
os.makedirs('/kaggle/working/data/val2017', exist_ok=True)

In [ ]:
import shutil
last_file = ''
for i, row in tqdm(df.iterrows(), total = len(df)):
    if row['file_path']!=last_file:
        last_file = row['file_path']
        base_dir = 'val2017' if row['fold']==0 else 'train2017'
        fname = f"{row['file_path']}"
        shutil.copyfile(f"{TRAIN_PATH}/{fname}", f"/kaggle/working/data/{base_dir}/{fname}")

## Sanity check

In [ ]:
!pip install -Uqqq 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

In [ ]:
from pycocotools.coco import COCO
import matplotlib.pyplot as plt
from PIL import Image
from random import sample

In [ ]:
data_dir = '/kaggle/working/data/train2017'
ann_file = '/kaggle/working/data/train.json'
coco = COCO(ann_file)
img_ids = coco.getImgIds()

In [ ]:
n_row = 2
n_col = 2
imgs = coco.loadImgs(sample(img_ids, n_row * n_col))
_, axs = plt.subplots(n_row, n_col, figsize=(12 * n_col, 8 * n_row))
axs = axs.flatten()
for img, ax in zip(imgs, axs):
    img_img = Image.open(f"{data_dir}/{img['file_name']}")
    anns = coco.loadAnns(coco.getAnnIds(imgIds=[img['id']]))
    ax.imshow(img_img)
    plt.sca(ax)
    coco.showAnns(anns, draw_bbox=True)
    plt.axis('off')
plt.show()

In [ ]:
shutil.make_archive('data', 'zip', 'data')

In [ ]:
shutil.rmtree('data') 